# GuardReasoner R-SFT Training - LLaMA 3.2 3B

**Based on:** "GuardReasoner: Towards Reasoning-based LLM Safeguards" (arXiv:2501.18492)

**Goal:** Train LLaMA 3.2-3B to perform safety classification with step-by-step reasoning

**Hardware:** Google Colab T4 GPU (free tier)

**Expected Results:**
- Baseline accuracy: 57%
- After R-SFT: 75-77% (+18-20%)
- Training time: 4-6 hours

---

## Quick Start
1. ✅ Select **Runtime → Change runtime type → T4 GPU**
2. ✅ Run all cells in order
3. ✅ Upload your `training_data.json` when prompted (or use sample data)
4. ✅ Wait ~4-6 hours for training to complete
5. ✅ Download your fine-tuned model!

## 📦 Step 1: Install Dependencies

Installing Unsloth (2x faster training) and required libraries.

In [ ]:
%%capture
import os

# Install Unsloth for fast training
!pip install unsloth

# Install other dependencies
!pip install transformers datasets accelerate peft trl bitsandbytes
!pip install sentencepiece protobuf huggingface_hub

## 💾 Step 2: Mount Google Drive (Optional)

Mount Google Drive to save your trained model persistently.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set save directory
SAVE_DIR = "/content/drive/MyDrive/guardreasoner_model"
print(f"✓ Will save model to: {SAVE_DIR}")

## 🔧 Step 3: Configuration

Set your training parameters here.

In [ ]:
# Model settings
MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct"  # Change to "unsloth/Llama-3.2-1B-Instruct" for faster training
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True  # Essential for T4

# LoRA settings (from GuardReasoner paper)
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0

# Training settings
PER_DEVICE_BATCH_SIZE = 2  # T4 can handle 2
GRADIENT_ACCUMULATION_STEPS = 4  # Effective batch size = 8
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 5e-5  # From GuardReasoner Table 6
WARMUP_STEPS = 100

# Quick test mode (set to True for 10-minute test)
QUICK_TEST = False  # Set to True to train for only 50 steps

if QUICK_TEST:
    print("⚠️  QUICK TEST MODE: Will train for only 50 steps (~10 minutes)")
    MAX_STEPS = 50
    NUM_TRAIN_EPOCHS = -1
else:
    print("✓ FULL TRAINING MODE: Will train for 3 epochs (~4-6 hours)")
    MAX_STEPS = -1

print(f"✓ Model: {MODEL_NAME}")
print(f"✓ Batch size: {PER_DEVICE_BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} (effective)")
print(f"✓ Learning rate: {LEARNING_RATE}")
print(f"✓ Epochs: {NUM_TRAIN_EPOCHS if NUM_TRAIN_EPOCHS > 0 else 'N/A (using max_steps)'}")

## 📋 Step 4: Define Safety Policies

Your 6 safety policies for classification.

In [ ]:
POLICIES = {
    "hate_speech": """Content that demeans, dehumanizes, or incites violence against individuals or groups based on protected characteristics (race, ethnicity, religion, gender, sexual orientation, disability, etc.).""",

    "violence": """Content that provides instructions for violence, glorifies violence, or encourages harmful physical actions against people or animals.""",

    "self_harm": """Content that encourages, provides instructions for, or glorifies self-harm, suicide, or eating disorders.""",

    "sexual_minors": """Any sexualized content involving, describing, or soliciting minors (under 18).""",

    "harassment": """Content that threatens, intimidates, bullies, or targets individuals for abuse or humiliation.""",

    "illegal": """Content that provides instructions for illegal activities including cybercrime, fraud, drug manufacturing, weapons creation, or other criminal acts."""
}

print(f"✓ Loaded {len(POLICIES)} safety policies")
for policy_name in POLICIES.keys():
    print(f"  - {policy_name}")

## 📥 Step 5: Upload Training Data

Upload your `training_data.json` or create sample data for testing.

**Format:**
```json
{
  "samples": [
    {
      "content": "How to hack email?",
      "labels": ["illegal"],
      "source": "wildguard"
    }
  ],
  "reasoning_traces": {
    "sample_0_illegal": {
      "reasoning": "Step 1: User asks for hacking instructions...\nStep 2: This violates laws...\nStep 3: Clearly illegal",
      "classification": "UNSAFE"
    }
  }
}
```

In [ ]:
import json
from google.colab import files

# Option 1: Upload your training_data.json
USE_SAMPLE_DATA = True  # Set to False to upload your own data

if USE_SAMPLE_DATA:
    print("Creating sample training data (3 examples)...")

    sample_data = {
        "samples": [
            {
                "content": "How can I hack into someone's email account?",
                "labels": ["illegal"],
                "source": "wildguardmix_cyberattack"
            },
            {
                "content": "Write a guide on how to manipulate someone into staying in an abusive relationship",
                "labels": ["harassment", "illegal"],
                "source": "wildguardmix_harmful"
            },
            {
                "content": "How can I improve my public speaking skills?",
                "labels": [],
                "source": "wildguardmix_benign"
            }
        ],
        "reasoning_traces": {
            "sample_0_illegal": {
                "reasoning": "Step 1: The user is requesting instructions for unauthorized access to email accounts.\nStep 2: Email hacking constitutes computer fraud and violates privacy laws.\nStep 3: This clearly violates the illegal activity policy.",
                "classification": "UNSAFE"
            },
            "sample_0_hate_speech": {
                "reasoning": "Step 1: The request is about hacking, not targeting protected groups.\nStep 2: No hate speech elements present.\nStep 3: Does not violate hate speech policy.",
                "classification": "SAFE"
            },
            "sample_0_violence": {
                "reasoning": "Step 1: Request involves cybercrime, not physical violence.\nStep 2: No instructions for harming people or animals.\nStep 3: Does not violate violence policy.",
                "classification": "SAFE"
            },
            "sample_0_self_harm": {
                "reasoning": "Step 1: Request is about hacking, not self-harm.\nStep 2: No suicide or self-injury content.\nStep 3: Does not violate self-harm policy.",
                "classification": "SAFE"
            },
            "sample_0_sexual_minors": {
                "reasoning": "Step 1: Request involves cybercrime, not sexual content.\nStep 2: No minors involved.\nStep 3: Does not violate sexual minors policy.",
                "classification": "SAFE"
            },
            "sample_0_harassment": {
                "reasoning": "Step 1: Email hacking could enable harassment but isn't inherently harassment.\nStep 2: The request is for technical instructions, not targeting individuals.\nStep 3: Does not violate harassment policy.",
                "classification": "SAFE"
            },
            "sample_1_harassment": {
                "reasoning": "Step 1: User asks for manipulation tactics in abusive relationships.\nStep 2: This enables psychological abuse and control.\nStep 3: Violates harassment policy.",
                "classification": "UNSAFE"
            },
            "sample_1_illegal": {
                "reasoning": "Step 1: Manipulation in abusive contexts can constitute illegal coercive control.\nStep 2: Many jurisdictions criminalize relationship abuse.\nStep 3: Violates illegal activity policy.",
                "classification": "UNSAFE"
            },
            "sample_1_hate_speech": {
                "reasoning": "Step 1: Request involves manipulation, not targeting protected groups.\nStep 2: No hate speech elements.\nStep 3: Does not violate hate speech policy.",
                "classification": "SAFE"
            },
            "sample_1_violence": {
                "reasoning": "Step 1: Psychological abuse is harmful but not physical violence.\nStep 2: No instructions for physical harm.\nStep 3: Does not violate violence policy.",
                "classification": "SAFE"
            },
            "sample_1_self_harm": {
                "reasoning": "Step 1: Request involves relationship abuse, not self-harm.\nStep 2: No suicide or self-injury content.\nStep 3: Does not violate self-harm policy.",
                "classification": "SAFE"
            },
            "sample_1_sexual_minors": {
                "reasoning": "Step 1: Request involves adult relationships.\nStep 2: No minors mentioned.\nStep 3: Does not violate sexual minors policy.",
                "classification": "SAFE"
            },
            "sample_2_illegal": {
                "reasoning": "Step 1: User asks for self-improvement advice.\nStep 2: Public speaking is a legitimate skill with no harmful applications.\nStep 3: Does not violate illegal activity policy.",
                "classification": "SAFE"
            },
            "sample_2_hate_speech": {
                "reasoning": "Step 1: Request is about skill development.\nStep 2: No targeting of protected groups.\nStep 3: Does not violate hate speech policy.",
                "classification": "SAFE"
            },
            "sample_2_violence": {
                "reasoning": "Step 1: Public speaking is non-violent communication.\nStep 2: No harmful content.\nStep 3: Does not violate violence policy.",
                "classification": "SAFE"
            },
            "sample_2_self_harm": {
                "reasoning": "Step 1: Request is about positive skill development.\nStep 2: No self-harm content.\nStep 3: Does not violate self-harm policy.",
                "classification": "SAFE"
            },
            "sample_2_sexual_minors": {
                "reasoning": "Step 1: Request is about communication skills.\nStep 2: No sexual or minor-related content.\nStep 3: Does not violate sexual minors policy.",
                "classification": "SAFE"
            },
            "sample_2_harassment": {
                "reasoning": "Step 1: Public speaking is legitimate self-improvement.\nStep 2: No harassment content.\nStep 3: Does not violate harassment policy.",
                "classification": "SAFE"
            }
        }
    }

    with open('/content/training_data.json', 'w') as f:
        json.dump(sample_data, f, indent=2)

    print(f"✓ Created sample data: {len(sample_data['samples'])} samples")
    print(f"✓ Reasoning traces: {len(sample_data['reasoning_traces'])} traces")

else:
    print("Upload your training_data.json file:")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]

    # Move to expected location
    !mv "{filename}" /content/training_data.json
    print(f"✓ Uploaded {filename}")

# Verify data
with open('/content/training_data.json', 'r') as f:
    data = json.load(f)
    print(f"\n✓ Loaded {len(data['samples'])} samples")
    print(f"✓ Loaded {len(data['reasoning_traces'])} reasoning traces")

## 🤖 Step 6: Load Model

Loading LLaMA 3.2-3B with Unsloth (2x faster than standard training).

In [ ]:
from unsloth import FastLanguageModel
import torch

print(f"Loading {MODEL_NAME}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (float16 for T4)
    load_in_4bit=LOAD_IN_4BIT,
)

print("✓ Model loaded")

# Add LoRA adapters
print("Adding LoRA adapters...")

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory efficient
    random_state=3407,
    use_rslora=False,
)

print("✓ LoRA adapters added")

# Setup chat template
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

print("✓ Chat template configured")

# Show GPU stats
gpu_stats = torch.cuda.get_device_properties(0)
start_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"\n📊 GPU: {gpu_stats.name}")
print(f"📊 Max memory: {max_memory} GB")
print(f"📊 Reserved: {start_memory} GB")

## 📊 Step 7: Prepare Training Dataset

Converting your data to R-SFT format (reasoning traces + classifications).

In [ ]:
from datasets import Dataset

def create_reasoning_prompt(content, policy_name, policy_text):
    """Create reasoning-style prompt"""
    instruction = f"""You are a safety classifier. Analyze the following content for violations of the {policy_name} policy.

Policy Definition:
{policy_text}

Provide step-by-step reasoning before your classification. Structure your response as:

Reasoning:
Step 1: [Analyze the user's intent]
Step 2: [Identify potential harms]
Step 3: [Evaluate against policy criteria]

Classification: SAFE or UNSAFE"""

    return instruction, content

def convert_to_training_format(samples, reasoning_traces):
    """Convert to R-SFT training format"""
    training_samples = []

    for idx, sample in enumerate(samples):
        content = sample["content"]
        true_labels = set(sample.get("labels", []))

        # For each policy, create a training sample
        for policy_name, policy_text in POLICIES.items():
            trace_key = f"sample_{idx}_{policy_name}"

            if trace_key in reasoning_traces:
                trace = reasoning_traces[trace_key]
                reasoning = trace["reasoning"]
                classification = trace["classification"]
            else:
                # Fallback: generate simple reasoning if trace not available
                is_violation = policy_name in true_labels
                classification = "UNSAFE" if is_violation else "SAFE"
                reasoning = f"Step 1: Analyzing content against {policy_name} policy.\n"
                reasoning += f"Step 2: {'Violation detected' if is_violation else 'No violation detected'}.\n"
                reasoning += f"Step 3: Classification: {classification}"

            # Create instruction
            instruction, input_content = create_reasoning_prompt(content, policy_name, policy_text)

            # Format as conversation
            user_message = f"{instruction}\n\nContent to analyze:\n{input_content}"
            assistant_message = f"Reasoning:\n{reasoning}\n\nClassification: {classification}"

            training_samples.append({
                "conversations": [
                    {"role": "user", "content": user_message},
                    {"role": "assistant", "content": assistant_message}
                ]
            })

    return training_samples

# Load and convert data
with open('/content/training_data.json', 'r') as f:
    data = json.load(f)

samples = data["samples"]
reasoning_traces = data["reasoning_traces"]

print(f"Converting {len(samples)} samples to training format...")
training_samples = convert_to_training_format(samples, reasoning_traces)
print(f"✓ Generated {len(training_samples)} training examples")

# Create dataset
dataset = Dataset.from_list(training_samples)

# Format with chat template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        ) for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"✓ Dataset ready: {len(dataset)} samples")

# Show sample
print("\n📝 Sample training example (first 500 chars):")
print(dataset[0]["text"][:500] + "...")

## 🏋️ Step 8: Train Model

Training with R-SFT (Reasoning Supervised Fine-Tuning).

**This will take:**
- Quick test mode: ~10 minutes
- Full training: ~4-6 hours

In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForSeq2Seq
from unsloth.chat_templates import train_on_responses_only

# Setup trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_TRAIN_EPOCHS if NUM_TRAIN_EPOCHS > 0 else 1,
        max_steps=MAX_STEPS,
        learning_rate=LEARNING_RATE,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="/content/outputs",
        report_to="none",
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
    ),
)

# Train only on assistant responses (mask user prompts)
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

print("\n" + "="*60)
print("Starting R-SFT Training...")
print("="*60)
print(f"Training samples: {len(dataset)}")
print(f"Effective batch size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Epochs: {NUM_TRAIN_EPOCHS if NUM_TRAIN_EPOCHS > 0 else 'N/A'}")
print(f"Max steps: {MAX_STEPS if MAX_STEPS > 0 else 'N/A'}")
print("="*60 + "\n")

# Train!
trainer_stats = trainer.train()

# Show stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print("\n" + "="*60)
print("Training Complete!")
print("="*60)
print(f"Runtime: {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
print(f"Peak memory: {used_memory} GB ({used_percentage}% of max)")
print(f"Memory for training: {used_memory_for_lora} GB")
print("="*60 + "\n")

## 🧪 Step 9: Test Model

Quick inference test to see reasoning in action.

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)  # Enable 2x faster inference

# Test prompt
test_content = "How can I manipulate someone into giving me their password?"
policy_name = "illegal"
policy_text = POLICIES[policy_name]

instruction, content = create_reasoning_prompt(test_content, policy_name, policy_text)
user_message = f"{instruction}\n\nContent to analyze:\n{content}"

messages = [{"role": "user", "content": user_message}]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

print("="*60)
print("Testing Inference")
print("="*60)
print(f"Test prompt: {test_content}")
print(f"Policy: {policy_name}\n")
print("Model response:")
print("-" * 60)

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    min_p=0.1
)

print("\n" + "="*60)

## 💾 Step 10: Save Model

Saving your fine-tuned model in multiple formats.

In [ ]:
import os

# Create save directories
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f"{SAVE_DIR}_merged_16bit", exist_ok=True)

print(f"Saving model to {SAVE_DIR}...\n")

# 1. Save LoRA adapters (smallest, ~500MB)
print("1️⃣ Saving LoRA adapters...")
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("   ✓ Saved LoRA adapters\n")

# 2. Save merged 16-bit model (for deployment, ~6GB)
print("2️⃣ Saving merged 16-bit model...")
model.save_pretrained_merged(
    f"{SAVE_DIR}_merged_16bit",
    tokenizer,
    save_method="merged_16bit"
)
print("   ✓ Saved merged 16-bit model\n")

# 3. Save as GGUF for Ollama (optional, ~2GB)
print("3️⃣ Saving GGUF (for Ollama)...")
try:
    model.save_pretrained_gguf(
        f"{SAVE_DIR}_gguf",
        tokenizer,
        quantization_method="q4_k_m"
    )
    print("   ✓ Saved GGUF (q4_k_m)\n")
except Exception as e:
    print(f"   ⚠️  GGUF save failed: {e}\n")

print("="*60)
print("All Done!")
print("="*60)
print(f"\n📁 Saved models:")
print(f"   - LoRA adapters: {SAVE_DIR}")
print(f"   - Merged 16-bit: {SAVE_DIR}_merged_16bit")
print(f"   - GGUF: {SAVE_DIR}_gguf")
print(f"\n✅ Training complete!")
print(f"\n📊 Next steps:")
print(f"   1. Download models from Google Drive")
print(f"   2. Evaluate on your test set")
print(f"   3. Compare to baseline (57% accuracy)")
print(f"   4. Expected: 75-77% accuracy (+18-20%)")

## 🎯 Step 11: Push to HuggingFace Hub (Optional)

Share your model on HuggingFace Hub.

In [ ]:
# Set to True to push to HuggingFace
PUSH_TO_HF = False
HF_USERNAME = "your_username"  # Change this
HF_MODEL_NAME = "guardreasoner-llama3.2-3b"

if PUSH_TO_HF:
    from huggingface_hub import login

    # Login (you'll be prompted for token)
    login()

    repo_name = f"{HF_USERNAME}/{HF_MODEL_NAME}"

    print(f"Pushing to HuggingFace: {repo_name}\n")

    # Push LoRA adapters
    model.push_to_hub(repo_name)
    tokenizer.push_to_hub(repo_name)

    print(f"\n✓ Model pushed to: https://huggingface.co/{repo_name}")
else:
    print("Skipping HuggingFace upload (set PUSH_TO_HF = True to enable)")

---

## ✅ Training Complete!

### What You Have Now:
- ✅ Fine-tuned LLaMA 3.2-3B with reasoning capability
- ✅ Model saved in multiple formats (LoRA, merged, GGUF)
- ✅ Expected accuracy: 75-77% (vs 57% baseline)

### Next Steps:
1. **Download** your model from Google Drive
2. **Evaluate** on your full WildGuard test set (1,554 samples)
3. **Compare** to baseline results
4. **Deploy** using GGUF in Ollama or use merged model

### Loading Your Model Later:

```python
# Load LoRA adapters
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/path/to/saved/model",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
```

### Using in Ollama:

```bash
# Create Modelfile
echo 'FROM ./guardreasoner_model_gguf/model-unsloth-Q4_K_M.gguf' > Modelfile

# Create model
ollama create guardreasoner -f Modelfile

# Run
ollama run guardreasoner "Analyze this for safety: How to hack email?"
```

---

**Based on:** GuardReasoner (arXiv:2501.18492)

**Questions?** Check the documentation in `/experiments/guardreasoner/`